In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

def get_news_content_by_date(target_date):
    # 目标网页URL
    url = "http://www.pbc.gov.cn/zhengcehuobisi/125207/125213/125440/3876551/index.html"

    # 发送HTTP请求获取网页内容
    response = requests.get(url)
    response.encoding = 'utf-8'  # 设置编码以正确解析中文内容

    # 使用BeautifulSoup解析网页内容
    soup = BeautifulSoup(response.text, 'html.parser')

    # 查找包含日期和内容的元素
    news_items = soup.find_all('tr')  # 根据实际情况调整选择器

    # 筛选出日期为目标日期的那条内容
    for item in news_items:
        date_span = item.find('span', class_='hui12')
        content_link = item.find('font', class_='newslist_style').find('a') if item.find('font', class_='newslist_style') else None
        
        if date_span and content_link:
            date_str = date_span.get_text(strip=True)
            
            # 检查日期是否匹配目标日期
            if date_str == target_date:
                content_text = content_link.get_text(strip=True)
                content_url = content_link['href']
                full_url = f"http://www.pbc.gov.cn{content_url}"
                print("日期：", date_str)
                print("内容：", content_text)
                print("网址：", full_url)
                
                # 发送HTTP请求获取新闻页面内容
                news_response = requests.get(full_url)
                news_response.encoding = 'utf-8'
                news_soup = BeautifulSoup(news_response.text, 'html.parser')
                
                # 提取class='zoom1'的内容
                zoom1_content = news_soup.find(class_='zoom1')
                if zoom1_content:
                    print("zoom1内容：", zoom1_content.get_text(strip=True))
                else:
                    print("未找到class='zoom1'的内容")
                return True
    return False

# 获取当前年份和月份
current_year = datetime.now().year
current_month = datetime.now().month

# 生成初始日期（20日）
target_dates = [
    f"{current_year}-{current_month:02d}-20",
    f"{current_year}-{current_month:02d}-21",
    f"{current_year}-{current_month:02d}-22"
]

# 尝试按顺序查找日期
found = False
for date in target_dates:
    if get_news_content_by_date(date):
        found = True
        break

# 如果没有找到，则提示用户输入日期
if not found:
    user_date = input("未找到相关日期内容，请输入目标日期（格式为YYYY-MM-DD）：")
    get_news_content_by_date(user_date)


日期： 2024-06-20
内容： 2024年6月20日全国银行间同业拆借中心受权公布贷款市场报价利率（LPR）公告
网址： http://www.pbc.gov.cn/zhengcehuobisi/125207/125213/125440/3876551/5380534/index.html
zoom1内容： 中国人民银行授权全国银行间同业拆借中心公布，2024年6月20日贷款市场报价利率（LPR）为：1年期LPR为3.45%，5年期以上LPR为3.95%。以上LPR在下一次发布LPR之前有效。
